In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers
!pip install flask-ngrok

In [ ]:
import pickle
from termcolor import colored
import torch
torch.cuda.empty_cache()
import pandas as pd
from transformers import RobertaTokenizer, RobertaModel, AdamW,RobertaForSequenceClassification
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, SequentialSampler
import numpy as np
import re
from flask import Flask, url_for, request, redirect
from flask_ngrok import run_with_ngrok
from IPython.display import Math, HTML, display

In [ ]:
robertaModel = RobertaForSequenceClassification.from_pretrained("roberta-base",num_labels = 2,output_attentions = False,output_hidden_states = False)
robertaTokenizer = RobertaTokenizer.from_pretrained("roberta-base")

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    robertaModel.cuda()
else:
    device = torch.device('cpu')
robertaModel.to(device)
print(device)

cuda:0


In [ ]:
robertaModel.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/OutputFiles/trainedmodel.pkl", map_location=device))

<All keys matched successfully>

In [ ]:
def formatText(title,content):
  title = title.strip()
  title = re.sub(' +', ' ', title)
  title = re.sub('[^A-Za-z0-9]+', ' ', title)
  title = title.lower()
  content = content.strip()
  content = re.sub(' +', ' ', content)
  content = re.sub('[^A-Za-z0-9]+', ' ', content)
  content = content.lower()
  finalText = title + ". " + content
  return finalText

In [ ]:
def getSentences(lenOfWords,lenToRemove):
  removeWordsFullList = []
  for i in range(lenOfWords-lenToRemove-1):
    removeWordsFullList.append([j for j in range(i,i+lenToRemove+1)])
  return removeWordsFullList

In [ ]:
def removeWordsFromSentence(words,removeWordsFullList):
  finalStrings = []
  for i in range(len(removeWordsFullList)):
    wordsCopy = words[:]
    for j in range(len(removeWordsFullList[i])):
      wordsCopy[removeWordsFullList[i][j]] = ""
    content = " ".join(wordsCopy)
    content = re.sub(' +', ' ', content)
    content = content.strip()
    finalStrings.append(content)
  return finalStrings

In [ ]:
def splitTextToIdedentifyTheImportantWords(finalText,remNum):
  words = finalText.split(" ")
  removeWordsFullList = getSentences(len(words),remNum)
  finalStrings = removeWordsFromSentence(words,removeWordsFullList)
  return finalStrings

In [ ]:
def convertToTensor(newsSentences,robertaTokenizer):
  ids = []
  masks = []
  batchSize = 16
  for news in newsSentences:
    encoded = robertaTokenizer.encode_plus(news,add_special_tokens = True,
                                            max_length = 256,
                                            pad_to_max_length = True,
                                            return_attention_mask = True,
                                            return_tensors = 'pt')
    ids.append(encoded['input_ids'])
    masks.append(encoded['attention_mask'])
  tensonDataset = TensorDataset(torch.cat(ids, dim=0), torch.cat(masks, dim=0))
  predictionSampler = SequentialSampler(tensonDataset)
  predictionDataloader = DataLoader(tensonDataset, sampler=predictionSampler, batch_size=batchSize)
  return predictionDataloader

In [ ]:
def getDifferenceBetweenPredictions(orginal,current):
  diff = abs(orginal[0]-current[0])
  diff = diff + abs(orginal[1]-current[1])
  return diff

In [ ]:
def checkOriginalTextStatus(finalText):
  robertaModel.eval()
  predictions = []
  finalText = " ".join(finalText.split(" ")[:256])
  highestDiffString = ""
  originalPrediction = []
  tensVal = convertToTensor([finalText],robertaTokenizer)
  count = 0
  newsStatus = ["FAKE","REAL"]
  for batch in tensVal:
    ids,masks = batch
    ids = ids.to(device)
    masks = masks.to(device)
    with torch.no_grad():
        outputs = robertaModel(ids, token_type_ids=None, 
                        attention_mask=masks)
    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    originalPrediction = logits[0]
  return newsStatus[np.argmax(originalPrediction)],originalPrediction

In [ ]:
def checkForMostImportantWords(finalText,originalPrediction):
  highestDiffVal = -float("inf")
  highestDiffString = ""
  maxLen = min(len(finalText.split(" "))/2,20)
  for checkWordCount in range(5,maxLen):
    allStringsToCheck = splitTextToIdedentifyTheImportantWords(finalText,checkWordCount)
    if(len(allStringsToCheck)>0):
      tensVal = convertToTensor(allStringsToCheck,robertaTokenizer)
      count = 0
      for batch in tensVal:
        ids,masks = batch
        ids = ids.to(device)
        masks = masks.to(device)
        with torch.no_grad():
            outputs = robertaModel(ids, token_type_ids=None, 
                            attention_mask=masks)
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        for logit in logits:
          diffrenceWithOriginal = getDifferenceBetweenPredictions(originalPrediction,logit)
          if(diffrenceWithOriginal > highestDiffVal):
            highestDiffString = allStringsToCheck[count]
            highestDiffVal = diffrenceWithOriginal
          count+=1
  return highestDiffString

In [ ]:
def highlightTheMostImportant(highestDiffString,finalText):
  diffStr = highestDiffString.split(" ")
  orginalStr = finalText.split(" ")
  diffStrCounter = 0
  orgStrCounter = 0
  newArr = []
  while(orgStrCounter < len(orginalStr) and diffStrCounter < len(diffStr)):
    if(orginalStr[orgStrCounter] != diffStr[diffStrCounter]):
      newArr.append("<mark>" + orginalStr[orgStrCounter]+"</mark>")
      orgStrCounter+=1
    else:
      newArr.append(orginalStr[orgStrCounter])
      diffStrCounter+=1
      orgStrCounter+=1
  return " ".join(newArr)

In [ ]:
def startCheckingForText(title,content):
  finalText = formatText(title,content)
  status,originalPrediction = checkOriginalTextStatus(finalText)
  highestDiffString = checkForMostImportantWords(finalText,originalPrediction)
  mostImportantSentence = highlightTheMostImportant(highestDiffString,finalText)
  return status,mostImportantSentence

In [ ]:
def set_css_in_cell_output():
  display(HTML("""<style>
  ol ol {
    list-style-type: lower-roman;
  }
  h1 {
    color: red;
  }
  </style>"""))

get_ipython().events.register('pre_run_cell', set_css_in_cell_output)

In [ ]:

app = Flask(__name__)
run_with_ngrok(app)   

@app.route('/login', methods=['GET', 'POST'])
def login():
  title = request.form['Title']
  content = request.form['Content']
  newsStatus,mostImportantSentence = startCheckingForText(title,content)
  return redirect(url_for('newsStatus',status = newsStatus,news = mostImportantSentence))

@app.route('/newsStatus/<status>')
def newsStatus(status):
  news = request.args.get('news')
  if(status == "REAL"):
    strToShow = '<body style="background-color:#99FF66;"><h1>The news is <span style="background-color: #99FF66">' + status +"</span></h1><br>"
  else:
    strToShow = '<body style="background-color:#FF6666;"><h1>The news is <span style="background-color: #FF6666">' + status +"</span></h1><br>"
  strToShow = strToShow + "Most Important words which influences the decision: <br><br>"+ news
  return strToShow

@app.route("/")
def home():
  reader = open('/content/drive/MyDrive/Colab Notebooks/templates/index.html','r')
  indexFile = reader.read()
  return indexFile
    
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://e2dc1af98148.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
